In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime

## Flightplan

In [ ]:
arr_df = pd.DataFrame()
dep_df = pd.DataFrame()

In [ ]:
daterange = pd.date_range(start='12/22/2019', end='12/28/2019').strftime('%Y%m%d')

In [ ]:
for date in daterange.tolist():
    arr = pd.read_excel(f'../data/flightplan/arr_{date}.xlsx', header = 0)
    dep = pd.read_excel(f'../data/flightplan/dep_{date}.xlsx', header = 0)
    datee = datetime.strptime(date,'%Y%m%d').strftime('%Y-%m-%d')
    arr['SDT'] = datee
    dep['SDT'] = datee
    arr_df = pd.concat([arr_df, arr])
    dep_df = pd.concat([dep_df, dep])

In [ ]:
arr_df = arr_df.drop('Seq', axis = 1)
dep_df = dep_df.drop('Seq', axis = 1)

In [ ]:
arr_df.to_csv('../input/arr_fp_20191222-20191228.csv')
dep_df.to_csv('../input/dep_fp_20191222-20191228.csv')

In [ ]:
# ACDM하고 데이터를 합칠 수 있을까?? -> 활주로 정보 넣기
# Traj0501 = io.loadmat(data_file)


## Takeoff Distance

T/O Distance 

- OAT, Pressure Altitude <br>
	-> METAR에서 가져올 수 있을 듯
	-> Density Altitude를 구하고 변수로 넣기

- Gross Weight <br>
	-> 이게 애매하단 말이지...
	-> 기종 별로는 할 수 있음

- Wind components <br>
	-> METAR에서 가져오기
	-> Headwind, Tailwind 성분만 가지고 하기

- RWY condition <br>
	-> METAR에서 가져오기
	-> Wet rwy 
	-> RWY slope는 rksi = 0


*** T/O speed varies with T/O Weight

In [ ]:
metar_data